# 51-形态学梯度（Morphology Gradient）

在进行大津二值化之后，计算图像的形态学梯度。

形态学梯度为经过膨胀操作（dilate）的图像与经过腐蚀操作（erode）的图像的差，可以用于抽出物体的边缘。

设置，形态学处理的核N=1。

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread("../picture/chans.png").astype(np.float32)

In [6]:
def BGR2GRAY(img):
    
    b = img[:,:,0]
    g = img[:,:,1]
    r = img[:,:,2]
    
    gray = 0.2126 * r + 0.7152 * g + 0.0722 * b
    gray = gray.astype(np.uint8)
    
    return gray

In [7]:
def otsu_binarization(img, th=128):
    
    H, W = img.shape
    out = img.copy()
    
    max_sigma = 0
    max_t = 0
    
    for t in range(1, 255):
        v0 = out[np.where(out < t)]
        m0 = np.mean(v0) if len(v0) > 0 else 0.
        w0 = len(v0) / (H * W)
        
        v1 = out[np.where(out >= t)]
        m1 = np.mean(v1) if len(v1) > 0 else 0.
        w1 = len(v1) / (H * W)
        
        sigma = w0 * w1 * ((m0 - m1) ** 2)
        
        if sigma > max_sigma:
            max_sigma = sigma
            max_t = t
            
    print("threshold >>", max_t)
    th = max_t
    out[out < th] = 0
    out[out >= th] = 255
    
    return out

In [4]:
def Dilate_code(img, time):
    
    H, W = img.shape
    
    m = np.array(((0, 1, 0), (1, 0, 1), (0, 1, 0)), dtype=np.int)
    
    out = img.copy()
    for i in range(time):
        tmp = np.pad(out, (1, 1), 'edge')
        for y in range(1, H+1):
            for x in range(1, W+1):
                if np.sum(m * tmp[y-1 : y+2, x-1 : x+2]) >= 255:
                    out[y-1, x-1] = 255
                    
    return out

In [5]:
def Erode_code(img, time):
    
    H, W = img.shape
    
    m = np.array(((0, 1, 0), (1, 0, 1), (0, 1, 0)), dtype=np.int)
    
    out = img.copy()
    for i in range(time):
        tmp = np.pad(out, (1, 1), 'edge')
        for y in range(1, H+1):
            for x in range(1, W+1):
                if np.sum(m * tmp[y-1 : y+2, x-1 : x+2]) < 255*4:
                    out[y-1, x-1] = 0
                    
    return out

In [8]:
gray = BGR2GRAY(img)

In [9]:
otsu = otsu_binarization(gray)

threshold >> 102


In [10]:
dilate = Dilate_code(otsu, time=1)

erode = Erode_code(otsu, time=1)

gradient = dilate - erode

In [12]:
cv2.imwrite('../picture/chan_result51_Gradient.jpg', gradient)
# cv2.namedWindow("result", 0);
# cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 52-顶帽（Top Hat）

在进行大津二值化之后，进行顶帽运算吧。

顶帽运算是原图像与开运算的结果图的差。

在这里，我们求大津二值化之后的图像和开处理（N=3）之后的图像的差，可以提取出细线状的部分或者噪声。

In [14]:
def opening(img, time):
    
    out = Dilate_code(img, time)
    out = Erode_code(out, time)
    
    return out

In [55]:
open_ = opening(otsu, time=3)

In [60]:
top = otsu - open_

In [53]:
H, W = open_.shape

top_hat = np.zeros((H, W), dtype=float)

for y in range(H):
    for x in range(W):
        top_hat[y, x] = otsu[y, x] - open_[y, x]

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in ubyte_scalars
  import sys


In [57]:
cv2.imwrite('../picture/chan_result52_TopHat.jpg', top_hat)
# cv2.namedWindow("result", 0);
# cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", top_hat)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 53-黑帽（Black Hat）

在进行大津二值化之后，进行黑帽运算吧。

黑帽运算是原图像与闭运算的结果图的差。

在这里，我们求大津二值化之后的图像和闭处理（N=3）之后的图像的差，在这里和顶帽运算一样，可以提取出细线状的部分或者噪声。

In [58]:
def closing(img, time):
    
    out = Erode_code(img, time)
    out = Dilate_code(out, time)
    
    return out

In [63]:
close = closing(otsu, time=3)

In [65]:
H, W = close.shape

black_hat = np.zeros((H, W), dtype=float)

for y in range(H):
    for x in range(W):
        black_hat[y, x] = otsu[y, x] - close[y, x]

In [66]:
cv2.imwrite('../picture/chan_result53_BlackHat.jpg', black_hat)
# cv2.namedWindow("result", 0);
# cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", black_hat)
cv2.waitKey(0)
cv2.destroyAllWindows()